<a href="https://colab.research.google.com/github/AndrewCrossman/Graduate_Research/blob/main/BNL_Scripts/XRD_File_Corrections.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# XRD File Corrections

This script helps make various corrections to XRD files in an xy format. All files must be located in an accessible google drive directory.

Current Corrections Available:

*   Background Subtraction




## Imports

In [3]:
from google.colab import drive  #allows downloading and accessing files on drive
from google.colab import files  #allows downloading and accessing files on drive
import shutil                   #allows uploading files to drive
import glob                     #useful for collecting and organizing files
import csv
import numpy as np              #math library
import pandas as pd             #data organization library
import scipy as sp              #math library
import re                       #string processing library
from scipy import optimize      #math fitting library
import matplotlib.pylab as plt  #allows for plotting
%pylab inline

Populating the interactive namespace from numpy and matplotlib


/usr/local/lib/python3.10/dist-packages/IPython/core/magics/pylab.py:159: UserWarning: pylab import has clobbered these variables: ['plt']
`%matplotlib` prevents importing * from pylab and numpy
  warn("pylab import has clobbered these variables: %s"  % clobbered +


## Extract Data from Google Drive

In [4]:
# log into google drive with your account
drive.mount('/content/drive/', force_remount=True)

Mounted at /content/drive/


In [24]:
# Directory Locations for information
XRD_files_path = '/content/drive/Shareddrives/BNL_June_2024/DataFiles/Cu1Ni2_200C/integration/'
XRD_tmps_path = '/content/drive/Shareddrives/BNL_June_2024/DataFiles/Cu1Ni2_200C/scalar_data/Cu1Ni2_mixed_LongRunInfo.csv'
XRD_subtraction_path = '/content/drive/Shareddrives/BNL_June_2024/DataFiles/Quartz_Capillary_no_FG_PDF_1sec/integration/'
XRD_subtraction_tmps_path = '/content/drive/Shareddrives/BNL_June_2024/DataFiles/Quartz_Capillary_no_FG_PDF_1sec/scalar_data/Quartz_Capillary_no_FG_PDF_1sec_LongRunInfo.csv'

In [25]:
# Read in XRD file temperature data
data1 = pd.read_csv(XRD_tmps_path)
data1 = data1
data1 = data1.sort_values(by='time')
# Convert eurotherm (i.e. control) temperature into sample temperature using a fit
C0,C1,C2,C3 = 4.91, 0.8519, 0.0005649, -2.536*10**(-7)
data1['sample_temp'] = round(C3*data1['eurotherm_temperature']**3 + C2*data1['eurotherm_temperature']**2 + C1*data1['eurotherm_temperature']**1 + C0,1)

# Read in XRD subtraction file temperature data
data2  = pd.read_csv(XRD_subtraction_tmps_path)
data2 = data2
data2 = data2.sort_values(by='time')
# Convert eurotherm (i.e. control) temperature into sample temperature using a fit
C0,C1,C2,C3 = 4.91, 0.8519, 0.0005649, -2.536*10**(-7)
data2['sample_temp'] = round(C3*data2['eurotherm_temperature']**3 + C2*data2['eurotherm_temperature']**2 + C1*data2['eurotherm_temperature']**1 + C0,1)

data1
data2

,time,pe1_stats1_total,eurotherm_temperature,eurotherm_power,chi_max,chi_argmax,gr_max,gr_argmax,seq_num,date,sample_temp
0,1.718247e+09,2.106761e+10,30.0,6.1,1.484337,618.14580,0.0,0.0,1,225655,31.0
1,1.718247e+09,2.106841e+10,30.0,7.6,1.484337,621.98883,0.0,0.0,2,225655,31.0
2,1.718247e+09,2.106894e+10,30.0,9.0,1.503549,623.33905,0.0,0.0,3,225655,31.0
3,1.718247e+09,2.106952e+10,31.0,10.7,1.484337,625.36830,0.0,0.0,4,225655,31.9
4,1.718247e+09,2.106991e+10,31.0,11.7,1.484337,627.42755,0.0,0.0,5,225655,31.9
...,...,...,...,...,...,...,...,...,...,...,...
463,1.718248e+09,2.107230e+10,464.0,44.8,1.484337,631.12310,0.0,0.0,113,230251,496.5
464,1.718248e+09,2.107229e+10,463.0,47.2,1.484337,629.37960,0.0,0.0,114,230251,495.3
465,1.718248e+09,2.107214e+10,463.0,46.7,1.484337,631.71783,0.0,0.0,115,230251,495.3
466,1.718248e+09,2.107180e+10,464.0,46.7,1.484337,628.12430,0.0,0.0,116,230251,496.5


In [26]:
XRD_files = glob.glob(XRD_files_path + '*.xy')                # collect all XRD data files in a list
XRD_sub_files = glob.glob(XRD_subtraction_path + '*.xy')      # collect all Quartz XRD files to subtract in a list
print(len(XRD_files), len(XRD_sub_files))                     # check the lengths of the two lists

2193 469


In [29]:
'''
Try and correlate the XRD files to the correct Quartz XRD file to subtract based on sample temperature.
Most but not necessarily all XRD files will have an associated Quartz file to subtract. In cases where
there are no associated Quartz file, a new file with the subtraction will not be created.
'''
for file in XRD_files[:30]:
  #new_name = file.replace(".xy","_quartz_subtraction.xy")
  XRD_data = pd.read_csv(file, skiprows=1, header=None, sep="\s+").astype(float)  # create a dataframe of the XRD data
  XRD_info = re.findall(r'\d+',file)                                              # find XRD identifier info encoded in file name
  XRD_temp = data1[(data1['seq_num']==float(XRD_info[-1])) &
                   (data1['date']==float(XRD_info[-4]))]['sample_temp'].item()        # find the sample temperature of the XRD file
  XRD_sub_file = data2[data2['sample_temp']==XRD_temp]                         # find the associated Quartz XRD with the same sample temperature (if none an empty dataframe will be returned)
  if XRD_sub_file.empty:                                                       # if the dataframe is empty skip the current XRD file and move to the next
    print("{} temp was skipped".format(XRD_temp))
    continue
  else:                                                                        # if the dataframe is not empty...
    XRD_sub_seq_num = XRD_sub_file.iloc[0]['seq_num'].item()                # find the associated Quartz file 'seq_num' identifier
    XRD_sub_date = XRD_sub_file.iloc[0]['date'].item()                         # find the associated Quartz file 'date' identifier
  XRD_sub_file_name = glob.glob(XRD_subtraction_path + '*{}*{}*.xy'.format(XRD_sub_date,XRD_sub_seq_num))  # find the filename of the XRD data to subtract
  print(XRD_subtraction_path + '*{}*{}*.xy'.format(XRD_sub_date,XRD_sub_seq_num))
  sub_data = pd.read_csv(XRD_sub_file_name[0], skiprows=1, header=None, sep="\s+").astype(float)              # create a dataframe of the XRD data to subtract

ValueError: can only convert an array of size 1 to a Python scalar